<center><h1>Kaggle

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import plotly.graph_objects as go
# import plotly.express as px
from sklearn.metrics import mean_absolute_percentage_error as mape # usage : print(mape(pred, observed))

import datetime

path = 'C:/Users/sebir/Desktop/M2 Toulouse/Défi IA/'

# Import data

In [2]:
# Training set, X
X = pd.read_csv(path + 'X_station_train.csv',
                parse_dates=['date'],
                infer_datetime_format=True)
X.columns = ['number_sta', 'date', 'wind_speed', 'temperature', 'dew_point_temperature', 'humidity', 'wind_direction', 'precipitation', 'Id']

# Training set, Y
Y = pd.read_csv(path + 'Y_train.csv',
                parse_dates=['date'],
                infer_datetime_format=True)

# Preprocess data

We have to group the data by data in order to have something comparable with `Y`

In [5]:
X['day'] = X['date'].apply(lambda x: x.date())

In [16]:
X_agg = X.copy()
X_agg = X_agg.groupby(by = ['day', 'number_sta'], as_index = False).agg({
    "number_sta"            : "first",
    "wind_speed"            : "median",
    "temperature"           : "median",
    "dew_point_temperature" : "median",
    "humidity"              : "median",
    "wind_direction"        : "median",
    "precipitation"         : "sum"
})

In [19]:
X_agg.isna().sum()

day                          0
number_sta                   0
wind_speed               72536
temperature               9047
dew_point_temperature    59029
humidity                 58933
wind_direction           72588
precipitation                0
dtype: int64

In [21]:
Y.isna().sum()

date                0
number_sta          0
Ground_truth    21640
Id                  0
dtype: int64

We firstly drop all the NAs.

In [22]:
X_agg.dropna(inplace = True)
Y = Y.loc[X_agg.index]

Y.dropna(inplace = True)
X_agg = X_agg.loc[Y.index]

We then delete the Id in `Y` which won't be helpful

In [25]:
X_agg.drop(['day'], axis = 1, inplace = True)
Y.drop(['date', 'Id'], axis = 1, inplace = True)

# Trial with Random Forest

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_agg, Y['Ground_truth'], test_size = 0.3)

In [29]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
y_predict = rf.predict(X_test)
print(f"Score: {mape(y_predict, y_test)}")

Score: 1.2691458866474445


We can try to first perform a scaler before doing the data:

In [32]:
X_agg

,number_sta,wind_speed,temperature,dew_point_temperature,humidity,wind_direction,precipitation
0,14066001,3.695,281.325,278.625,90.15,142.0,0.2
2,14137001,5.830,281.155,278.905,93.50,140.0,0.0
3,14216001,5.010,280.630,278.815,91.25,155.5,0.6
7,14372001,5.650,280.600,278.465,89.45,158.5,0.2
11,14578001,5.230,280.195,278.760,95.90,160.0,0.8
...,...,...,...,...,...,...,...
183739,86027001,8.590,286.140,285.065,90.85,240.0,5.9
183742,86137003,3.765,286.450,284.880,88.90,227.5,3.4
183743,86165005,6.265,286.110,284.320,87.65,245.5,5.6
183745,91200002,4.380,286.070,284.555,93.45,233.5,5.6


In [39]:
from sklearn.preprocessing import scale
X_scaled = scale(X_agg.drop('number_sta', axis = 1)) # centralisation et normalisation
X_scaled = pd.DataFrame(X_scaled, columns = X_agg.drop('number_sta', axis = 1).columns)
X_scaled['number_sta'] = X_agg['number_sta'].to_list()

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y['Ground_truth'], test_size = 0.3)
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)
print(f"Score: {mape(y_predict, y_test)}")

Score: 1.427041928987862


# Trial with XGBRegressor

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_agg, Y['Ground_truth'], test_size = 0.3)
from sklearn.ensemble import GradientBoostingRegressor
rf = GradientBoostingRegressor()
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)
print(f"Score: {mape(y_predict, y_test)}")

Score: 1.3802698501187114


# Submission

In [65]:
X_test = pd.read_csv(path + 'X_station_test.csv')
X_test

,dd,hu,td,t,ff,precip,month,Id
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8
...,...,...,...,...,...,...,...,...
2304797,190.0,82.8,277.00,279.74,10.62,0.0,12,95690001_176_19
2304798,195.0,84.2,277.44,279.93,11.86,0.0,12,95690001_176_20
2304799,199.0,85.7,277.95,280.21,11.77,0.0,12,95690001_176_21
2304800,198.0,85.3,278.25,280.58,10.16,0.0,12,95690001_176_22
